In [ ]:
import os
from joblib import dump, load
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.lines import Line2D
import cartopy.crs as ccrs
import warnings
import matplotlib.colors as mcolors
import numpy as np

### Set the initial conditions

In [ ]:
# Open your dataset for wind gust
path = '/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_gust_data/ICON-CH1-EPS_2024-07-11T21_agg.nc'
ds_icon_gust = xr.open_dataset(path)  # Replace with your actual dataset

# Open your dataset for precipitation
path = '/scratch2/cwenger/ICON-CH1_monthly/ICON-CH1_07_2024_agg.nc'
ds_icon_precip = xr.open_dataset(path)  # Replace with your actual dataset

# Date
selected_date = '2024071121'
# Leadtime
selected_leadtime = 12

# Campground name
c_name = 'Lugano'

# Wind gust thresholds in m s⁻¹
gust_threshold_1 = 18
gust_threshold_2 = 25

# Precipitation thresholds in mm h⁻¹
precip_threshold_1 = 21.4
precip_threshold_2 = 47.8


### Create a dataframe for wind gust with all campgrounds and the corresponding coordinates and nearest grid cells

In [ ]:
# Define the locations and their coordinates
locations = {
    "Bern": (2595842.15, 1201548.69),
    "Brienz": (2646563.43, 1177603.88),
    "Buochs": (2674514.65, 1203568.08),
    "Interlaken": (2632995.62, 1171399.67),
    "Lugano": (2713926.15, 1094747.56),
    "Salavaux": (2569175.20, 1195846.24),
    "Sempach": (2657011.60, 1219718.50),
    "Gordevio": (2700590.96, 1119701.60),
    "Thun": (2614554.68, 1178201.33),
    "Morges": (2527116.10, 1150785.39)
}

# Loop through each location to find the nearest x and y and save variables
for name, (loc_x, loc_y) in locations.items():
    # Find the nearest grid cell
    nearest_x = ds_icon_gust.sel(x=loc_x, y=loc_y, method="nearest").x.values
    nearest_y = ds_icon_gust.sel(x=loc_x, y=loc_y, method="nearest").y.values
    # Save variables dynamically
    globals()[f"{name}_cell_x"] = nearest_x
    globals()[f"{name}_cell_y"] = nearest_y


locations_df = pd.DataFrame([(name, coord[0], coord[1]) for name, coord in locations.items()], columns=['name', 'x_coordinate', 'y_coordinate'])
# Add x_cell and y_cell columns to the dataframe
locations_df['x_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_x"))
locations_df['y_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_y"))
dump(locations_df, 'campgrounds_coordinates_gust.pkl')

# Load the campground dataframe and select subset of dataset that only consider the chosen campground
campgrounds_gust = load('campgrounds_coordinates_gust.pkl')

### Create a dataframe for precipitaton with all campgrounds and the corresponding coordinates and nearest grid cells

In [ ]:
# Define the locations and their coordinates
locations = {
    "Bern": (2595842.15, 1201548.69),
    "Brienz": (2646563.43, 1177603.88),
    "Buochs": (2674514.65, 1203568.08),
    "Interlaken": (2632995.62, 1171399.67),
    "Lugano": (2713926.15, 1094747.56),
    "Salavaux": (2569175.35, 1195846.24),
    "Sempach": (2657011.60, 1219718.50),
    "Gordevio": (2700590.96, 1119701.60),
    "Thun": (2614554.68, 1178351.33),
    "Morges": (2527116.10, 1150785.39)
}

# Loop through each location to find the nearest x and y and save variables
for name, (loc_x, loc_y) in locations.items():
    # Find the nearest grid cell
    nearest_x = ds_icon_precip.sel(x=loc_x, y=loc_y, method="nearest").x.values
    nearest_y = ds_icon_precip.sel(x=loc_x, y=loc_y, method="nearest").y.values

    # Save variables dynamically
    globals()[f"{name}_cell_x"] = nearest_x
    globals()[f"{name}_cell_y"] = nearest_y


locations_df = pd.DataFrame([(name, coord[0], coord[1]) for name, coord in locations.items()], columns=['name', 'x_coordinate', 'y_coordinate'])
# Add x_cell and y_cell columns to the dataframe
locations_df['x_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_x"))
locations_df['y_cell_nearest'] = locations_df['name'].apply(lambda name: globals().get(f"{name}_cell_y"))
dump(locations_df, 'campgrounds_coordinates_precip.pkl')

# Load the campground dataframe and select subset of dataset that only consider the chosen campground
campgrounds_precip = load('campgrounds_coordinates_precip.pkl')

#### Choose campground for wind gust plot

In [ ]:
# Define the window size (in grid points)
full_window = 35
half_window = int((full_window-1)/2)

center_x = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
center_y = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['y_cell_nearest'].values[0]

# Get the indices of the nearest grid point
x_index = int(ds_icon_gust.x.to_index().get_loc(center_x))
y_index = int(ds_icon_gust.y.to_index().get_loc(center_y))

# Compute index bounds
x_start_idx = max(x_index - half_window, 0)
x_end_idx = min(x_index + half_window + 1, ds_icon_gust.sizes['x'])
y_start_idx = max(y_index - half_window, 0)
y_end_idx = min(y_index + half_window + 1, ds_icon_gust.sizes['y'])

# Extract the window for gust data
window_icon_gust = ds_icon_gust.isel(x=slice(x_start_idx, x_end_idx), y=slice(y_start_idx, y_end_idx))

Window_icon now contains only the gust data for the chosen campground with a 35kmx35km window around it

#### Plot wind gust for the specific campground and the window around the campground

In [ ]:
warnings.filterwarnings("ignore")

# plot the actual data, for a specific date, time step, and member
for member_number in range(11): # Loop through members 0 to 10
    data_wind = window_icon_gust.sel(lead_time=selected_leadtime, realization=member_number).wind_speed_of_gust.squeeze()

    # Define the Swiss coordinate system (EPSG:2056)
    swiss_proj = ccrs.epsg(2056)

    # Compute the extent of the data
    min_x, max_x = data_wind.x.values.min(), data_wind.x.values.max()
    min_y, max_y = data_wind.y.values.min(), data_wind.y.values.max()
    extent_x = max_x - min_x
    extent_y = max_y - min_y

    # Dynamically adjust figure size to maintain a square aspect ratio
    fig_size = 13  # Base figure size
    aspect_ratio = extent_y / extent_x
    fig_width = fig_size
    fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
    fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

    # Create the figure and axes
    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = plt.axes(projection=swiss_proj)
    ax.set_aspect('1')  # Ensure square aspect ratio for the map

    # Set the extent of the map
    ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)


    # Add the WMS layer for Switzerland's topography
    wms_url = 'https://wms.geo.admin.ch/?'
    layer = 'ch.swisstopo.pixelkarte-grau'
    ax.add_wms(wms_url, layer)

    # Add the WMS layer for Switzerland's borders
    border_layer = 'ch.swisstopo.swissboundaries3d-land-flaeche.fill'
    ax.add_wms(wms_url, layers=border_layer)
    canton_layer = 'ch.swisstopo.swissboundaries3d-kanton-flaeche.fill'  # Cantonal borders


    # Define colormap and normalization
    colors = [
    "#F9EEC3",  # 18–21
    "#E3CF7A",  # 21–24
    "#B7A34B",  # 24–27
    "#7A763A",  # 27–30
    "#444B2B",  # >30
    ]
    cmap = mcolors.ListedColormap(colors)
    bounds = [18, 21, 24, 27, 30]
    norm = mcolors.BoundaryNorm(bounds, cmap.N - 1)


    # Plot the data
    im = ax.contourf(
        data_wind.x,
        data_wind.y,
        data_wind,
        levels=bounds,
        transform=swiss_proj,
        cmap=cmap, norm=norm,
        alpha=0.8, extend='max'
    )

    # add blue contour line for threshold 1
    cs = ax.contour(
        data_wind.x,
        data_wind.y,
        data_wind,
        levels=[gust_threshold_1],
        colors='#1E90FF',
        linewidths=4.0,
        transform=swiss_proj,
        linestyles='-'
    )

    # add red contour line for threshold 2
    cs = ax.contour(
        data_wind.x,
        data_wind.y,
        data_wind,
        levels=[gust_threshold_2],
        colors='#D62828',
        linewidths=4.0,
        transform=swiss_proj,
        linestyles='-'
    )

    # Add legend
    legend_lines = [
        Line2D([0], [0], color="#1E90FF", lw=4.0),
        Line2D([0], [0], color="#D62828", lw=4.0),
        ]
    legend_labels = [f'{gust_threshold_1} m s⁻¹', f'{gust_threshold_2} m s⁻¹']
    legend = ax.legend(
        handles=legend_lines,
        labels=legend_labels,
        loc='lower left',        # alignment
        alignment="left",        # text alignment
        bbox_to_anchor=(0.245, -0.407),  # slightly below the plot area
        fontsize=35,
        framealpha=0.9,
        title='Warning Threshold',
        title_fontsize=35
)

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.0)

    # Add the colorbar
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.02, aspect=30, shrink=0.825)
    cbar.set_label('Wind Gust [m s⁻¹]', fontsize=35)
    cbar.set_ticks(bounds)
    cbar.ax.yaxis.set_major_locator(mticker.FixedLocator(bounds))
    cbar.set_ticklabels([str(b) for b in bounds])
    cbar.ax.tick_params(labelsize=35)

    # Add dot where camping site is located
    # Plot campsite location as a prominent dot on top of all layers
    x_dot = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
    y_dot = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['y_cell_nearest'].values[0]
    ax.scatter(
        x_dot, y_dot,
        transform=swiss_proj,
        s=250,              
        c='#FF2E2E',      
        edgecolors='k',       
        linewidths=1.5,
        zorder=1000
    )

    # Extract date, time, and member values
    date_val = str(selected_date)
    time_val = int(data_wind.coords['lead_time'].values)
    member_val = int(data_wind.coords['realization'].values)
    c_name_lower = c_name[0].lower() + c_name[1:]


    # Save figure
    directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
    plt.savefig(f'{directory_to_save}/{date_val}_{c_name_lower}_gust_time{time_val}_member{member_val}.png', dpi=150, bbox_inches='tight')
    


#### Choose campground for precipitation plot

In [ ]:
# Define the window size (in grid points)
full_window = 35
half_window = int((full_window-1)/2)

center_x = campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
center_y = campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['y_cell_nearest'].values[0]

# Get the indices of the nearest grid point
x_index = int(ds_icon_precip.x.to_index().get_loc(center_x))
y_index = int(ds_icon_precip.y.to_index().get_loc(center_y))

# Compute index bounds
x_start_idx = max(x_index - half_window, 0)
x_end_idx = min(x_index + half_window + 1, ds_icon_precip.sizes['x'])
y_start_idx = max(y_index - half_window, 0)
y_end_idx = min(y_index + half_window + 1, ds_icon_precip.sizes['y'])

# Extract the window for precipitation data
window_icon_precip = ds_icon_precip.isel(x=slice(x_start_idx, x_end_idx), y=slice(y_start_idx, y_end_idx))

Window_icon now contains only the precipitation data for the chosen campground with a 35kmx35km window around it

#### Plot the precipitation for the specific campground and the window around the campground

In [ ]:
warnings.filterwarnings("ignore")

# Plot the actual data, for a specific date, time step, and member
for member_number in range(11): # Loop through members 0 to 10
    data_prec = window_icon_precip.sel(date=selected_date, time=selected_leadtime, members=member_number).tot_prec_1h

    # Define the Swiss coordinate system (EPSG:2056)
    swiss_proj = ccrs.epsg(2056)

    # Compute the extent of the data
    min_x, max_x = data_prec.x.values.min(), data_prec.x.values.max()
    min_y, max_y = data_prec.y.values.min(), data_prec.y.values.max()
    extent_x = max_x - min_x
    extent_y = max_y - min_y

    # Dynamically adjust figure size to maintain a square aspect ratio
    fig_size = 13  # Base figure size
    aspect_ratio = extent_y / extent_x
    fig_width = fig_size
    fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
    fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

    # Create the figure and axes
    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = plt.axes(projection=swiss_proj)
    ax.set_aspect('1')  # Ensure square aspect ratio for the map

    # Set the extent of the map
    ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)


    # Add the WMS layer for Switzerland's topography
    wms_url = 'https://wms.geo.admin.ch/?'
    layer = 'ch.swisstopo.pixelkarte-grau'
    ax.add_wms(wms_url, layer)

    # Add the WMS layer for Switzerland's borders
    border_layer = 'ch.swisstopo.swissboundaries3d-land-flaeche.fill'
    ax.add_wms(wms_url, layers=border_layer)
    canton_layer = 'ch.swisstopo.swissboundaries3d-kanton-flaeche.fill'  # Cantonal borders


    # Define colormap and normalization
    colors = [
    '#7B9FC4',  # 10–20 
    '#5F88B8',  # 20–30
    '#486EA9',  # 30–40
    '#3B5592',  # 40–50
    '#333C78',  # 50–60
    '#2C2461'   # >60
    ]
    cmap = mcolors.ListedColormap(colors)
    bounds = [10, 20, 30, 40, 50, 60]
    norm = mcolors.BoundaryNorm(bounds, cmap.N - 1)



    # Plot the data
    im = ax.contourf(
        data_prec.x,
        data_prec.y,
        data_prec,
        levels=bounds,
        transform=swiss_proj,
        cmap=cmap, norm=norm,
        alpha=0.8, extend='max'
    )

    # add yellow contour line for threshold 1
    cs = ax.contour(
        data_prec.x,
        data_prec.y,
        data_prec,
        levels=[precip_threshold_1],
        colors="#FFAE00",
        linewidths=4.0,
        transform=swiss_proj,
        linestyles='-'
    )

    # add red contour line for threshold 2
    cs = ax.contour(
        data_prec.x,
        data_prec.y,
        data_prec,
        levels=[precip_threshold_2],
        colors='#D62828',
        linewidths=4.0,
        transform=swiss_proj,
        linestyles='-'
    )

    # Add legend
    legend_lines = [
        Line2D([0], [0], color="#FFAE00", lw=4.0),
        Line2D([0], [0], color="#D62828", lw=4.0),
    ]
    legend_labels = [f'{precip_threshold_1} mm h⁻¹', f'{precip_threshold_2} mm h⁻¹']
    legend = ax.legend(
        handles=legend_lines,
        labels=legend_labels,
        loc='lower left',        # alignment
        alignment="left",        # text alignment
        bbox_to_anchor=(0.225, -0.407),  # slightly below the plot area
        fontsize=35,
        framealpha=0.9,
        title='Warning Threshold',
        title_fontsize=35
    )
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.0)

    # Add the colorbar
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.02, aspect=30, shrink=0.825)
    cbar.set_label('Precipitation [mm h⁻¹]', fontsize=35)
    cbar.ax.tick_params(labelsize=35)
    
    # Add dot where camping site is located
    # Plot campsite location as a prominent dot on top of all layers
    x_dot = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
    y_dot = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['y_cell_nearest'].values[0]
    ax.scatter(
        x_dot, y_dot,
        transform=swiss_proj,
        s=250,              
        c='#FF2E2E',      
        edgecolors='k',       
        linewidths=1.5,
        zorder=1000
    )
    
    # Extract date, time, and member values
    date_val = str(data_prec.coords['date'].values)
    time_val = int(data_prec.coords['time'].values)
    member_val = int(data_prec.coords['members'].values)
    c_name_lower = c_name[0].lower() + c_name[1:]

    # Save figure
    directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
    plt.savefig(f'{directory_to_save}/{date_val}_{c_name_lower}_precipitation_time{time_val}_member{member_val}.png', dpi=150, bbox_inches='tight')



### Wind gust contours plotted for members exceeding threshold levels

In [ ]:
warnings.filterwarnings("ignore")

data_wind = window_icon_gust.sel(lead_time=selected_leadtime).wind_speed_of_gust.squeeze()

# Define the Swiss coordinate system (EPSG:2056)
swiss_proj = ccrs.epsg(2056)

# Compute the extent of the data
min_x, max_x = data_wind.x.values.min(), data_wind.x.values.max()
min_y, max_y = data_wind.y.values.min(), data_wind.y.values.max()
extent_x = max_x - min_x
extent_y = max_y - min_y

# Dynamically adjust figure size to maintain a square aspect ratio
fig_size = 13  # Base figure size
aspect_ratio = extent_y / extent_x
fig_width = fig_size
fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

# Define colormap and normalization
colors = [
    "#1f77b4", "#fd6500", "#2ca02c", "#d62728",
    "#9467bd", "#8c564b", "#e377c2", "#4c136d",
    "#bcbd22", "#17becf", "#f1a707"
]
cmap = mcolors.ListedColormap(colors)

for thr in (gust_threshold_1, gust_threshold_2):
    # create a fresh figure/axes for each threshold to avoid overlaying plots/legends
    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = plt.axes(projection=swiss_proj)
    ax.set_aspect('1')
    ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)

    # fresh legend lists per threshold
    legend_lines = []
    legend_labels = []

    for member_number in range(11):
        data_wind_m = window_icon_gust.sel(
            lead_time=selected_leadtime,
            realization=member_number
        ).wind_speed_of_gust.squeeze()

        max_val = float(data_wind_m.max())
        if max_val <= thr:
            continue

        ax.contourf(
            data_wind_m.x, data_wind_m.y, data_wind_m,
            levels=[thr, max_val],
            colors=[colors[member_number]],
            alpha=0.3,
            transform=swiss_proj
        )
        ax.contour(
            data_wind_m.x, data_wind_m.y, data_wind_m,
            levels=[thr],
            colors=[colors[member_number]],
            linewidths=4.0,
            linestyles="-",
            transform=swiss_proj
        )
        legend_lines.append(Line2D([0], [0], color=colors[member_number], lw=4.0))
        legend_labels.append(f"Member {member_number}")

    if legend_lines:
        handles_ordered = legend_lines[:6] + legend_lines[6:]
        labels_ordered  = legend_labels[:6] + legend_labels[6:]

        legend = ax.legend(
            handles=handles_ordered,
            labels=labels_ordered,
            ncol=2,
            loc='upper left',
            bbox_to_anchor=(0.01, 0.00),
            bbox_transform=ax.transAxes,
            fontsize=35,
            framealpha=0.9,
            title=f'Wind Gust Contours for {thr} m s⁻¹',
            title_fontsize=35,
            columnspacing=2.0,
            handlelength=2.2,
            handletextpad=0.6,
            labelspacing=0.4,
        )
        legend._legend_box.align = "left"
    else:
        ax.text(0.5, -0.05, f'No members exceed {thr} m s⁻¹', transform=ax.transAxes,
                ha='center', va='top', fontsize=20)

    ax.set_title(f'Members Exceeding Wind Gust\nThreshold of {thr} m s⁻¹', fontsize=35, pad=11)

    # campsite dot
    x_dot = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
    y_dot = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['y_cell_nearest'].values[0]
    ax.scatter(
        x_dot, y_dot,
        transform=swiss_proj,
        s=250,
        c='#FF2E2E',
        edgecolors='k',
        linewidths=1.5,
        zorder=1000
    )

    # ensure directory exists and save
    c_name_lower = c_name[0].lower() + c_name[1:]
    directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
    os.makedirs(directory_to_save, exist_ok=True)
    filename = f"{directory_to_save}/{selected_date}_{c_name_lower}_gust_members_exceeding_threshold{thr}_time{time_val}.png"
    plt.savefig(filename, dpi=150, bbox_inches="tight")


### Precipitation contours plotted for members exceeding threshold level

In [ ]:
warnings.filterwarnings("ignore")

data_precip = window_icon_precip.sel(time=selected_leadtime).tot_prec_1h.squeeze()

# Define the Swiss coordinate system (EPSG:2056)
swiss_proj = ccrs.epsg(2056)

# Compute the extent of the data
min_x, max_x = data_precip.x.values.min(), data_precip.x.values.max()
min_y, max_y = data_precip.y.values.min(), data_precip.y.values.max()
extent_x = max_x - min_x
extent_y = max_y - min_y

# Dynamically adjust figure size to maintain a square aspect ratio
fig_size = 13  # Base figure size
aspect_ratio = extent_y / extent_x
fig_width = fig_size
fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

# Define colormap and normalization
colors = [
    "#1f77b4", "#fd6500", "#2ca02c", "#d62728",
    "#9467bd", "#8c564b", "#e377c2", "#4c136d",
    "#bcbd22", "#17becf", "#f1a707"
]
cmap = mcolors.ListedColormap(colors)

for thr in (precip_threshold_1, precip_threshold_2):
    # create a fresh figure/axes for each threshold to avoid overlaying plots/legends
    fig = plt.figure(figsize=(fig_width, fig_height))
    ax = plt.axes(projection=swiss_proj)
    ax.set_aspect('1')
    ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)

    # fresh legend lists per threshold
    legend_lines = []
    legend_labels = []

    for member_number in range(11):
        data_precip_m = window_icon_precip.sel(
            date=selected_date,
            time=selected_leadtime,
            members=member_number
        ).tot_prec_1h.squeeze()

        max_val = float(data_precip_m.max())
        if max_val <= thr:
            continue

        ax.contourf(
            data_precip_m.x, data_precip_m.y, data_precip_m,
            levels=[thr, max_val],
            colors=[colors[member_number]],
            alpha=0.3,
            transform=swiss_proj
        )
        ax.contour(
            data_precip_m.x, data_precip_m.y, data_precip_m,
            levels=[thr],
            colors=[colors[member_number]],
            linewidths=4.0,
            linestyles="-",
            transform=swiss_proj
        )
        legend_lines.append(Line2D([0], [0], color=colors[member_number], lw=4.0))
        legend_labels.append(f"Member {member_number}")

    if legend_lines:
        handles_ordered = legend_lines[:6] + legend_lines[6:]
        labels_ordered  = legend_labels[:6] + legend_labels[6:]

        legend = ax.legend(
            handles=handles_ordered,
            labels=labels_ordered,
            ncol=2,
            loc='upper left',
            bbox_to_anchor=(0.01, 0.00),
            bbox_transform=ax.transAxes,
            fontsize=35,
            framealpha=0.9,
            title=f'Precipitation Contours for {thr} mm h⁻¹',
            title_fontsize=35,
            columnspacing=2.0,
            handlelength=2.2,
            handletextpad=0.6,
            labelspacing=0.4,
        )
        legend._legend_box.align = "left"
    else:
        ax.text(0.5, -0.05, f'No members exceed {thr} m s⁻¹', transform=ax.transAxes,
                ha='center', va='top', fontsize=20)

    ax.set_title(f'Members Exceeding Precipitation Threshold \nof {thr} mm h⁻¹', fontsize=35, pad=11)

    # campsite dot
    x_dot = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
    y_dot = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['y_cell_nearest'].values[0]
    ax.scatter(
        x_dot, y_dot,
        transform=swiss_proj,
        s=250,
        c='#FF2E2E',
        edgecolors='k',
        linewidths=1.5,
        zorder=1000
    )

    # ensure directory exists and save
    c_name_lower = c_name[0].lower() + c_name[1:]
    directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
    os.makedirs(directory_to_save, exist_ok=True)
    filename = f"{directory_to_save}/{selected_date}_{c_name_lower}_precipitation_members_exceeding_threshold{thr}_time{time_val}.png"
    plt.savefig(filename, dpi=150, bbox_inches="tight")


### Calculate the probabilities for wind gust and precipitation that a certain threshold is exceeded and plot a probability map

#### Wind gust

In [ ]:
# compute probability of exceedance across ensemble members

# Select leadtime
window_icon_gust_time = window_icon_gust.sel(lead_time=selected_leadtime)

# first, we look at each grid point in the window and check whether the threshold is exceeded
exceedance_thres_1 = (window_icon_gust_time.wind_speed_of_gust > gust_threshold_1)
exceedance_thres_2 = (window_icon_gust_time.wind_speed_of_gust > gust_threshold_2)

# we then count how many members at each grid point exceed the threshold
count_exceedance_thres_1 = exceedance_thres_1.sum(dim='realization')
count_exceedance_thres_2 = exceedance_thres_2.sum(dim='realization')

# and then we divide by the total number of members to get the probability
probability_exceedance_gust_thres_1 = (count_exceedance_thres_1 / window_icon_gust_time.sizes['realization'] * 100).squeeze()
probability_exceedance_gust_thres_2 = (count_exceedance_thres_2 / window_icon_gust_time.sizes['realization'] * 100).squeeze()

Plot the probability map for wind gust threshold 1

In [ ]:
warnings.filterwarnings("ignore")

# extent and figure sizing 
min_x, max_x = float(probability_exceedance_gust_thres_1.x.min().item()), float(probability_exceedance_gust_thres_1.x.max().item())
min_y, max_y = float(probability_exceedance_gust_thres_1.y.min().item()), float(probability_exceedance_gust_thres_1.y.max().item())
extent_x = max_x - min_x
extent_y = max_y - min_y

fig_size = 13
aspect_ratio = extent_y / extent_x if extent_x != 0 else 1.0
fig_width = fig_size
fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

fig = plt.figure(figsize=(fig_width, fig_height))
swiss_proj = ccrs.epsg(2056)
ax = plt.axes(projection=swiss_proj)
ax.set_aspect('1')
ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)

# background WMS
wms_url = 'https://wms.geo.admin.ch/?'
ax.add_wms(wms_url, 'ch.swisstopo.pixelkarte-grau')
ax.add_wms(wms_url, layers='ch.swisstopo.swissboundaries3d-land-flaeche.fill')

# Colormap
alpha = 0.75

# Convert alpha to two-digit hex (00–FF)
alpha_hex = format(int(alpha * 255), "02X")

# Base MeteoSwiss colors (without alpha)
gust_hex = [
    "#FFFFFF",  # 0 - 10%
    "#FFF6E0",  # 10 - 20%
    "#F9EEC3",  # 20 - 30%
    "#EAD684",  # 30 - 40%
    "#D8C05A",  # 40 - 50%
    "#BFA347",  # 50 - 60%
    "#9E8840",  # 60 - 70%
    "#7E6F38",  # 70 - 80%
    "#5E5831",  # 80 - 90%
    "#444B2B",  # 90 -100%
]

# Add transparency to all except the first
gust_hex_alpha = [gust_hex[0] + "00"] + [c + alpha_hex for c in gust_hex[1:]]

cmap = mcolors.ListedColormap(gust_hex_alpha)

# 10-step boundaries 0,10,20,...,100
levels = np.arange(0, 101, 10)

# BoundaryNorm for discrete bins (clip=True to avoid out-of-range issues)
norm = mcolors.BoundaryNorm(levels, ncolors=cmap.N, clip=True)

# Set plotting alpha (0.0 transparent .. 1.0 opaque)
plot_alpha = 0.75

# Plot discrete probability map

im = ax.contourf(
    probability_exceedance_gust_thres_1.x,
    probability_exceedance_gust_thres_1.y,
    probability_exceedance_gust_thres_1,
    levels=levels, cmap=cmap, norm=norm,
    transform=swiss_proj, extend='neither'
)

cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.02, aspect=30, shrink=0.825, ticks=levels)
cbar.set_label('Exceedance Probability [%]', fontsize=35)
cbar.ax.tick_params(labelsize=35)
ax.set_title(f'Wind Gust of {gust_threshold_1} m s⁻¹', fontsize=35, pad=11)

# add red contour line where probability exceeds 73%
threshold_line = (8/11)*100

cs = ax.contour(
    probability_exceedance_gust_thres_1.x,
    probability_exceedance_gust_thres_1.y,
    probability_exceedance_gust_thres_1,
    levels=[threshold_line],
    colors='#D62828',
    linewidths=4.0,
    transform=swiss_proj,
    linestyles='-'
)

# Add legend
legend_lines = [Line2D([0], [0], color="#D62828", lw=4.0)]
legend_labels = [f'73%']
legend = ax.legend(
    handles=legend_lines,
    labels=legend_labels,
    loc='lower left',        # alignment
    alignment="left",        # text alignment
    bbox_to_anchor=(0.245, -0.355),  # slightly below the plot area
    fontsize=35,
    framealpha=0.9,
    title='Warning Threshold',
    title_fontsize=35
)

plt.tight_layout()
plt.subplots_adjust(bottom=0.0)

# Add dot where camping site is located
# Plot campsite location as a prominent dot on top of all layers
x_dot = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
y_dot = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['y_cell_nearest'].values[0]
ax.scatter(
    x_dot, y_dot,
    transform=swiss_proj,
    s=250,              
    c='#FF2E2E',      
    edgecolors='k',       
    linewidths=1.5,
    zorder=1000
)

# save
c_name_lower = c_name[0].lower() + c_name[1:]
date_val = str(selected_date)
time_val = int(selected_leadtime)
directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
os.makedirs(directory_to_save, exist_ok=True)
outfn = f'{directory_to_save}/{date_val}_{c_name_lower}_probability_gust_above{gust_threshold_1}_time{time_val}_over_all_members.png'
plt.savefig(outfn, dpi=150, bbox_inches='tight')


Plot the probability map for wind gust threshold 2

In [ ]:
warnings.filterwarnings("ignore")

# extent and figure sizing 
min_x, max_x = float(probability_exceedance_gust_thres_2.x.min().item()), float(probability_exceedance_gust_thres_2.x.max().item())
min_y, max_y = float(probability_exceedance_gust_thres_2.y.min().item()), float(probability_exceedance_gust_thres_2.y.max().item())
extent_x = max_x - min_x
extent_y = max_y - min_y

fig_size = 13
aspect_ratio = extent_y / extent_x if extent_x != 0 else 1.0
fig_width = fig_size
fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

fig = plt.figure(figsize=(fig_width, fig_height))
swiss_proj = ccrs.epsg(2056)
ax = plt.axes(projection=swiss_proj)
ax.set_aspect('1')
ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)

# background WMS
wms_url = 'https://wms.geo.admin.ch/?'
ax.add_wms(wms_url, 'ch.swisstopo.pixelkarte-grau')
ax.add_wms(wms_url, layers='ch.swisstopo.swissboundaries3d-land-flaeche.fill')

# Colormap
alpha = 0.75

# Convert alpha to two-digit hex (00–FF)
alpha_hex = format(int(alpha * 255), "02X")

# Base MeteoSwiss colors (without alpha)
gust_hex = [
    "#FFFFFF",  # 0 - 10%
    "#FFF6E0",  # 10 - 20%
    "#F9EEC3",  # 20 - 30%
    "#EAD684",  # 30 - 40%
    "#D8C05A",  # 40 - 50%
    "#BFA347",  # 50 - 60%
    "#9E8840",  # 60 - 70%
    "#7E6F38",  # 70 - 80%
    "#5E5831",  # 80 - 90%
    "#444B2B",  # 90 -100%
]

# Add transparency to all except the first
gust_hex_alpha = [gust_hex[0] + "00"] + [c + alpha_hex for c in gust_hex[1:]]

cmap = mcolors.ListedColormap(gust_hex_alpha)

# 10-step boundaries 0,10,20,...,100
levels = np.arange(0, 101, 10)

# BoundaryNorm for discrete bins (clip=True to avoid out-of-range issues)
norm = mcolors.BoundaryNorm(levels, ncolors=cmap.N, clip=True)

# Set plotting alpha (0.0 transparent .. 1.0 opaque)
plot_alpha = 0.75

# Plot discrete probability map

im = ax.contourf(
    probability_exceedance_gust_thres_2.x,
    probability_exceedance_gust_thres_2.y,
    probability_exceedance_gust_thres_2,
    levels=levels, cmap=cmap, norm=norm,
    transform=swiss_proj, extend='neither'
)

cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.02, aspect=30, shrink=0.825, ticks=levels)
cbar.set_label('Exceedance Probability [%]', fontsize=35)
cbar.ax.tick_params(labelsize=35)
ax.set_title(f'Wind Gust of {gust_threshold_2} m s⁻¹', fontsize=35, pad=11)

# add red contour line where probability exceeds 73%
threshold_line = (8/11)*100

cs = ax.contour(
    probability_exceedance_gust_thres_2.x,
    probability_exceedance_gust_thres_2.y,
    probability_exceedance_gust_thres_2,
    levels=[threshold_line],
    colors='#D62828',
    linewidths=4.0,
    transform=swiss_proj,
    linestyles='-'
)

# Add legend
legend_lines = [Line2D([0], [0], color="#D62828", lw=4.0)]
legend_labels = [f'73%']
legend = ax.legend(
    handles=legend_lines,
    labels=legend_labels,
    loc='lower left',        # alignment
    alignment="left",        # text alignment
    bbox_to_anchor=(0.245, -0.355),  # slightly below the plot area
    fontsize=35,
    framealpha=0.9,
    title='Warning Threshold',
    title_fontsize=35
)

plt.tight_layout()
plt.subplots_adjust(bottom=0.0)

# Add dot where camping site is located
# Plot campsite location as a prominent dot on top of all layers
x_dot = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
y_dot = campgrounds_gust[campgrounds_gust['name'].str.contains(c_name)]['y_cell_nearest'].values[0]
ax.scatter(
    x_dot, y_dot,
    transform=swiss_proj,
    s=250,              
    c='#FF2E2E',      
    edgecolors='k',       
    linewidths=1.5,
    zorder=1000
)

# save
c_name_lower = c_name[0].lower() + c_name[1:]
date_val = str(selected_date)
time_val = int(selected_leadtime)
directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
os.makedirs(directory_to_save, exist_ok=True)
outfn = f'{directory_to_save}/{date_val}_{c_name_lower}_probability_gust_above{gust_threshold_2}_time{time_val}_over_all_members.png'
plt.savefig(outfn, dpi=150, bbox_inches='tight')


#### Precipitation

In [ ]:
# compute probability of exceedance across ensemble members

# Select leadtime
window_icon_precip_time = window_icon_precip.sel(date=selected_date, time=selected_leadtime)

# first, we look at each grid point in the window and check whether the threshold is exceeded

exceedance_thres_1 = (window_icon_precip_time.tot_prec_1h > precip_threshold_1)
exceedance_thres_2 = (window_icon_precip_time.tot_prec_1h > precip_threshold_2)

# we then count how many members at each grid point exceed the threshold
count_exceedance_thres_1 = exceedance_thres_1.sum(dim='members')
count_exceedance_thres_2 = exceedance_thres_2.sum(dim='members')

# and then we divide by the total number of members to get the probability
probability_exceedance_precip_thres_1 = count_exceedance_thres_1 / window_icon_precip_time.sizes['members'] * 100
probability_exceedance_precip_thres_2 = count_exceedance_thres_2 / window_icon_precip_time.sizes['members'] * 100

Plot the probability map for precipitation threshold 1

In [ ]:
from matplotlib.lines import Line2D

warnings.filterwarnings("ignore")

# extent and figure sizing 
min_x, max_x = float(probability_exceedance_precip_thres_1.x.min().item()), float(probability_exceedance_precip_thres_1.x.max().item())
min_y, max_y = float(probability_exceedance_precip_thres_1.y.min().item()), float(probability_exceedance_precip_thres_1.y.max().item())
extent_x = max_x - min_x
extent_y = max_y - min_y

fig_size = 13
aspect_ratio = extent_y / extent_x if extent_x != 0 else 1.0
fig_width = fig_size
fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

fig = plt.figure(figsize=(fig_width, fig_height))
swiss_proj = ccrs.epsg(2056)
ax = plt.axes(projection=swiss_proj)
ax.set_aspect('1')
ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)

# background WMS
wms_url = 'https://wms.geo.admin.ch/?'
ax.add_wms(wms_url, 'ch.swisstopo.pixelkarte-grau')
ax.add_wms(wms_url, layers='ch.swisstopo.swissboundaries3d-land-flaeche.fill')

# Colormap
alpha = 0.75

# Convert alpha to two-digit hex (00–FF)
alpha_hex = format(int(alpha * 255), "02X")

# Base MeteoSwiss colors (without alpha)
meteoswiss_hex = [
    "#ffffff",  # 0 - 10%
    "#f5fbf5",  # 10 - 20%
    "#dff7e8",  # 20 - 30%
    "#bff0cc",  # 30 - 40%
    "#98eeb0",  # 40 - 50%
    "#59d79a",  # 50 - 60%
    "#7fdfe6",  # 60 - 70%
    "#58a8db",  # 70 - 80%
    "#2b79b6",  # 80 - 90%
    "#123f5a",  # 90 -100%
]

# Add transparency to all except the first
meteoswiss_hex_alpha = [meteoswiss_hex[0] + "00"] + [c + alpha_hex for c in meteoswiss_hex[1:]]

cmap = mcolors.ListedColormap(meteoswiss_hex_alpha)

# 10-step boundaries 0,10,20,...,100
levels = np.arange(0, 101, 10)

# BoundaryNorm for discrete bins (clip=True to avoid out-of-range issues)
norm = mcolors.BoundaryNorm(levels, ncolors=cmap.N, clip=True)

# Set plotting alpha (0.0 transparent .. 1.0 opaque)
plot_alpha = 0.75

# Plot discrete probability map

im = ax.contourf(
    probability_exceedance_precip_thres_1.x,
    probability_exceedance_precip_thres_1.y,
    probability_exceedance_precip_thres_1,
    levels=levels, cmap=cmap, norm=norm,
    transform=swiss_proj, extend='neither'
)

cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.02, aspect=30, shrink=0.825, ticks=levels)
cbar.set_label('Exceedance Probability [%]', fontsize=35)
cbar.ax.tick_params(labelsize=35)
ax.set_title(f'Precipitation of {precip_threshold_1} mm h⁻¹', fontsize=35, pad=11)

# add red contour line where probability exceeds 73%
threshold_line = (8/11)*100

cs = ax.contour(
    probability_exceedance_precip_thres_1.x,
    probability_exceedance_precip_thres_1.y,
    probability_exceedance_precip_thres_1,
    levels=[threshold_line],
    colors='#D62828',
    linewidths=4.0,
    transform=swiss_proj,
    linestyles='-'
)
# Add legend
legend_lines = [Line2D([0], [0], color="#D62828", lw=4.0)]
legend_labels = ['73%']

legend = fig.legend(
    handles=legend_lines,
    labels=legend_labels,
    loc='lower left',        # alignment
    alignment="left",        # text alignment
    bbox_to_anchor=(0.3, -0.12),  # slightly below the plot area
    fontsize=35,
    framealpha=1.0,
    title='Warning Threshold',
    title_fontsize=35
)

plt.tight_layout()
plt.subplots_adjust(bottom=0.0)

# Add dot where camping site is located
# Plot campsite location as a prominent dot on top of all layers
x_dot = campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
y_dot = campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['y_cell_nearest'].values[0]
ax.scatter(
    x_dot, y_dot,
    transform=swiss_proj,
    s=250,              
    c='#FF2E2E',      
    edgecolors='k',       
    linewidths=1.5,
    zorder=1000
)

# save
c_name_lower = c_name[0].lower() + c_name[1:]
date_val = str(selected_date)
time_val = int(selected_leadtime)
directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
os.makedirs(directory_to_save, exist_ok=True)
outfn = f'{directory_to_save}/{date_val}_{c_name_lower}_probability_precipitation_above{precip_threshold_1}_time{time_val}_over_all_members.png'
plt.savefig(outfn, dpi=150, bbox_inches='tight')


Plot the probability map for precipitation threshold 2

In [ ]:
warnings.filterwarnings("ignore")

# extent and figure sizing 
min_x, max_x = float(probability_exceedance_precip_thres_2.x.min().item()), float(probability_exceedance_precip_thres_2.x.max().item())
min_y, max_y = float(probability_exceedance_precip_thres_2.y.min().item()), float(probability_exceedance_precip_thres_2.y.max().item())
extent_x = max_x - min_x
extent_y = max_y - min_y

fig_size = 13
aspect_ratio = extent_y / extent_x if extent_x != 0 else 1.0
fig_width = fig_size
fig_height = fig_size * aspect_ratio if aspect_ratio >= 1 else fig_size
fig_width = fig_size / aspect_ratio if aspect_ratio < 1 else fig_size

fig = plt.figure(figsize=(fig_width, fig_height))
swiss_proj = ccrs.epsg(2056)
ax = plt.axes(projection=swiss_proj)
ax.set_aspect('1')
ax.set_extent([min_x, max_x, min_y, max_y], crs=swiss_proj)

# background WMS
wms_url = 'https://wms.geo.admin.ch/?'
ax.add_wms(wms_url, 'ch.swisstopo.pixelkarte-grau')
ax.add_wms(wms_url, layers='ch.swisstopo.swissboundaries3d-land-flaeche.fill')

# Colormap
alpha = 0.75

# Convert alpha to two-digit hex (00–FF)
alpha_hex = format(int(alpha * 255), "02X")

# Base MeteoSwiss colors (without alpha)
meteoswiss_hex = [
    "#ffffff",  # 0 - 10%
    "#f5fbf5",  # 10 - 20%
    "#dff7e8",  # 20 - 30%
    "#bff0cc",  # 30 - 40%
    "#98eeb0",  # 40 - 50%
    "#59d79a",  # 50 - 60%
    "#7fdfe6",  # 60 - 70%
    "#58a8db",  # 70 - 80%
    "#2b79b6",  # 80 - 90%
    "#123f5a",  # 90 -100%
]

# Add transparency to all except the first
meteoswiss_hex_alpha = [meteoswiss_hex[0] + "00"] + [c + alpha_hex for c in meteoswiss_hex[1:]]

cmap = mcolors.ListedColormap(meteoswiss_hex_alpha)

# 10-step boundaries 0,10,20,...,100
levels = np.arange(0, 101, 10)

# BoundaryNorm for discrete bins (clip=True to avoid out-of-range issues)
norm = mcolors.BoundaryNorm(levels, ncolors=cmap.N, clip=True)

# Set plotting alpha (0.0 transparent .. 1.0 opaque)
plot_alpha = 0.75

# Plot discrete probability map

im = ax.contourf(
    probability_exceedance_precip_thres_2.x,
    probability_exceedance_precip_thres_2.y,
    probability_exceedance_precip_thres_2,
    levels=levels, cmap=cmap, norm=norm,
    transform=swiss_proj, extend='neither'
)

cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.02, aspect=30, shrink=0.825, ticks=levels)
cbar.set_label('Exceedance Probability [%]', fontsize=35)
cbar.ax.tick_params(labelsize=35)
ax.set_title(f'Precipitation of {precip_threshold_2} mm h⁻¹', fontsize=35, pad=11)

# add red contour line where probability exceeds 73%
threshold_line = (8/11)*100

cs = ax.contour(
    probability_exceedance_precip_thres_2.x,
    probability_exceedance_precip_thres_2.y,
    probability_exceedance_precip_thres_2,
    levels=[threshold_line],
    colors='#D62828',
    linewidths=4.0,
    transform=swiss_proj,
    linestyles='-'
)
# Add legend
legend_lines = [Line2D([0], [0], color="#D62828", lw=4.0)]
legend_labels = ['73%']

legend = fig.legend(
    handles=legend_lines,
    labels=legend_labels,
    loc='lower left',        # alignment
    alignment="left",        # text alignment
    bbox_to_anchor=(0.3, -0.12),  # slightly below the plot area
    fontsize=35,
    framealpha=1.0,
    title='Warning Threshold',
    title_fontsize=35
)

plt.tight_layout()
plt.subplots_adjust(bottom=0.0)

# Add dot where camping site is located
# Plot campsite location as a prominent dot on top of all layers
x_dot = campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['x_cell_nearest'].values[0]
y_dot = campgrounds_precip[campgrounds_precip['name'].str.contains(c_name)]['y_cell_nearest'].values[0]
ax.scatter(
    x_dot, y_dot,
    transform=swiss_proj,
    s=250,              
    c='#FF2E2E',      
    edgecolors='k',       
    linewidths=1.5,
    zorder=1000
)

# save
c_name_lower = c_name[0].lower() + c_name[1:]
date_val = str(selected_date)
time_val = int(selected_leadtime)
directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
os.makedirs(directory_to_save, exist_ok=True)
outfn = f'{directory_to_save}/{date_val}_{c_name_lower}_probability_precipitation_above{precip_threshold_2}_time{time_val}_over_all_members.png'
plt.savefig(outfn, dpi=150, bbox_inches='tight')


### Print statistics for a selected date, leadtime, and member

In [ ]:
print(f"\n--- Some statistics for {c_name} at {selected_date}, leadtime {selected_leadtime} ---")

# Define thresholds for wind damage
threshold_1 = 18
threshold_2 = 25

# Save the area numbers for each member
area_thresholds_1, area_thresholds_2 = [], []

# Over one specific member
for member in range(11):
    print(f"\n---  Member {member} ---")
    wind_gust = window_icon_gust.sel(lead_time=selected_leadtime, realization=member).wind_speed_of_gust
    precipitation = window_icon_precip.sel(date=selected_date, time=selected_leadtime, members=member).tot_prec_1h
    max_wind_gust_member = wind_gust.max().values
    min_wind_gust_member = wind_gust.min().values
    mean_wind_gust_member = wind_gust.mean().values
    max_precipitation_member = precipitation.max().values
    min_precipitation_member = precipitation.min().values
    mean_precipitation_member = precipitation.mean().values

    # Save the area numbers for each member
    area_thresholds_1, area_thresholds_2 = [], []

    data_wind = window_icon_gust.sel(
        lead_time=selected_leadtime,
        realization=member
    ).wind_speed_of_gust.squeeze()

    exceed_count_1 = int(((data_wind > threshold_1).sum(dim=["y", "x"])).item())

    data_wind = window_icon_gust.sel(
        lead_time=selected_leadtime,
        realization=member
    ).wind_speed_of_gust.squeeze()

    exceed_count_2 = int(((data_wind > threshold_2).sum(dim=["y", "x"])).item())

    print("Max. wind gust:", max_wind_gust_member, "m s⁻¹")
    print("Min. wind gust:", min_wind_gust_member, "m s⁻¹")
    print("Mean wind gust:", mean_wind_gust_member, "m s⁻¹")
    print()
    print("Max. precipitation:", max_precipitation_member, "mm h⁻¹")
    print("Min. precipitation:", min_precipitation_member, "mm h⁻¹")
    print("Mean precipitation:", mean_precipitation_member, "mm h⁻¹")
    print()
    print(f"Area exceeding wind threshold of {threshold_1} m s⁻¹:", exceed_count_1, "km² of 1225 km² (", round(exceed_count_1 / 1225 * 100), "% )")
    print(f"Area exceeding wind threshold of {threshold_2} m s⁻¹:", exceed_count_2, "km² of 1225 km² (", round(exceed_count_2 / 1225 * 100), "% )")

# Save the area numbers for each member
area_thresholds_1, area_thresholds_2 = [], []
print(f"\n--- Areas exceeding wind threshold of {threshold_1} m s⁻¹ (per member) ---")
for member_number in range(11):
    data_wind = window_icon_gust.sel(
        lead_time=selected_leadtime,
        realization=member_number
    ).wind_speed_of_gust.squeeze()

    exceed_count_1 = int(((data_wind > threshold_1).sum(dim=["y", "x"])).item())
    area_thresholds_1.append(exceed_count_1)
    
    print(f"Member {member_number}: {exceed_count_1} km² of 1225 km² ({round(exceed_count_1 / 1225 * 100)}%)")
print()
print(f"--- Areas exceeding wind threshold of {threshold_2} m s⁻¹ (per member) ---")
for member_number in range(11):
    data_wind = window_icon_gust.sel(
        lead_time=selected_leadtime,
        realization=member_number
    ).wind_speed_of_gust.squeeze()
    
    exceed_count_2 = int(((data_wind > threshold_2).sum(dim=["y", "x"])).item())
    area_thresholds_2.append(exceed_count_2)

    print(f"Member {member_number}: {exceed_count_2} km² of 1225 km² ({round(exceed_count_2 / 1225 * 100)}%)")

# Over all members
max_wind_gust = []
min_wind_gust = []
mean_wind_gust = []
max_precipitation = []
min_precipitation = []
mean_precipitation = []
mean_area_threshold_1 = []
mean_area_threshold_2 = []

for member in range(11):
    wind = window_icon_gust.sel(lead_time=selected_leadtime, realization=member).wind_speed_of_gust
    precipitation = window_icon_precip.sel(date=selected_date, time=selected_leadtime, members=member).tot_prec_1h
    max_wind_gust.append(wind.max().values)
    min_wind_gust.append(wind.min().values)
    mean_wind_gust.append(wind.mean().values)
    max_precipitation.append(precipitation.max().values)
    min_precipitation.append(precipitation.min().values)
    mean_precipitation.append(precipitation.mean().values)
    mean_area_threshold_1.append(np.mean(area_thresholds_1))
    mean_area_threshold_2.append(np.mean(area_thresholds_2))

print("\n--- Over all 11 members ---")
print("Max. wind gust:", np.max(max_wind_gust), "m s⁻¹")
print("Min. wind gust:", np.min(min_wind_gust), "m s⁻¹")
print("Mean wind gust:", np.mean(mean_wind_gust), "m s⁻¹")
print()
print("Max. precipitation:", np.max(max_precipitation), "mm h⁻¹")
print("Min. precipitation:", np.min(min_precipitation), "mm h⁻¹")
print("Mean precipitation:", np.mean(mean_precipitation), "mm h⁻¹")
print()
print(f"Mean area exceeding wind threshold of {threshold_1} m s⁻¹: {round(np.mean(mean_area_threshold_1))} km² of 1225 km² ({round(np.mean(mean_area_threshold_1) / 1225 * 100)}%)")
print(f"Mean area exceeding wind threshold of {threshold_2} m s⁻¹: {round(np.mean(mean_area_threshold_2))} km² of 1225 km² ({round(np.mean(mean_area_threshold_2) / 1225 * 100)}%)")

# Save statistics to a text file
c_name_lower = c_name[0].lower() + c_name[1:]
directory_to_save = f'/scratch2/cwenger/ICON-CH1_monthly/figures_{c_name_lower}'
with open(f'{directory_to_save}/{selected_date}_{c_name_lower}_gust_precipitation_statistics_time{selected_leadtime}.txt', 'w') as f:
    f.write(f"\n--- Some statistics for {c_name} at {selected_date}, leadtime {selected_leadtime} ---\n")
    for member in range(11):
        wind_gust = window_icon_gust.sel(lead_time=selected_leadtime, realization=member).wind_speed_of_gust
        precipitation = window_icon_precip.sel(date=selected_date, time=selected_leadtime, members=member).tot_prec_1h
        max_wind_gust_member = wind_gust.max().values
        min_wind_gust_member = wind_gust.min().values
        mean_wind_gust_member = wind_gust.mean().values
        max_precipitation_member = precipitation.max().values
        min_precipitation_member = precipitation.min().values
        mean_precipitation_member = precipitation.mean().values
        
        # Save the area numbers for each member
        area_thresholds_1, area_thresholds_2 = [], []

        data_wind = window_icon_gust.sel(
            lead_time=selected_leadtime,
            realization=member
        ).wind_speed_of_gust.squeeze()

        exceed_count_1 = int(((data_wind > threshold_1).sum(dim=["y", "x"])).item())

        data_wind = window_icon_gust.sel(
            lead_time=selected_leadtime,
            realization=member
        ).wind_speed_of_gust.squeeze()

        exceed_count_2 = int(((data_wind > threshold_2).sum(dim=["y", "x"])).item())

        f.write(f"\n--- Member {member} ---\n")
        f.write(f"Max. wind gust: {max_wind_gust_member} m s⁻¹\n")
        f.write(f"Min. wind gust: {min_wind_gust_member} m s⁻¹\n")
        f.write(f"Mean wind gust: {mean_wind_gust_member} m s⁻¹\n")
        f.write(f"\n")
        f.write(f"Max. precipitation: {max_precipitation_member} mm h⁻¹\n")
        f.write(f"Min. precipitation: {min_precipitation_member} mm h⁻¹\n")
        f.write(f"Mean precipitation: {mean_precipitation_member} mm h⁻¹\n")
        f.write(f"\n")
        f.write(f"Area exceeding wind threshold of {threshold_1} m s⁻¹: {exceed_count_1} km² of 1225 km² ({round(exceed_count_1 / 1225 * 100)}%)\n")
        f.write(f"Area exceeding wind threshold of {threshold_2} m s⁻¹: {exceed_count_2} km² of 1225 km² ({round(exceed_count_2 / 1225 * 100)}%)\n")

    f.write(f"\n--- Areas exceeding wind threshold of {threshold_1} m s⁻¹ (per member) ---\n")
    for member_number in range(11):
        data_wind = window_icon_gust.sel(
            lead_time=selected_leadtime,
            realization=member_number
        ).wind_speed_of_gust.squeeze()

        exceed_count_1 = int(((data_wind > threshold_1).sum(dim=["y", "x"])).item())
        area_thresholds_1.append(exceed_count_1)

        f.write(f"Member {member_number}: {exceed_count_1} km² of 1225 km² ({round(exceed_count_1 / 1225 * 100)}%)\n")
    f.write(f"\n--- Areas exceeding wind threshold of {threshold_2} m s⁻¹ (per member) ---\n")
    for member_number in range(11):
        data_wind = window_icon_gust.sel(
            lead_time=selected_leadtime,
            realization=member_number
        ).wind_speed_of_gust.squeeze()

        exceed_count_2 = int(((data_wind > threshold_2).sum(dim=["y", "x"])).item())
        area_thresholds_2.append(exceed_count_2)

        f.write(f"Member {member_number}: {exceed_count_2} km² of 1225 km² ({round(exceed_count_2 / 1225 * 100)}%)\n")


    f.write(f"\n--- Over all 11 members ---\n")
    f.write(f"Max. wind gust: {np.max(max_wind_gust)} m s⁻¹\n")
    f.write(f"Min. wind gust: {np.min(min_wind_gust)} m s⁻¹\n")
    f.write(f"Mean wind gust: {np.mean(mean_wind_gust)} m s⁻¹\n")
    f.write(f"\n")
    f.write(f"Max. precipitation: {np.max(max_precipitation)} mm h⁻¹\n")
    f.write(f"Min. precipitation: {np.min(min_precipitation)} mm h⁻¹\n")
    f.write(f"Mean precipitation: {np.mean(mean_precipitation)} mm h⁻¹\n")
    f.write(f"\n")
    f.write(f"Mean area exceeding wind threshold of {threshold_1} m s⁻¹: {round(np.mean(mean_area_threshold_1))} km² of 1225 km² ({round(np.mean(mean_area_threshold_1) / 1225 * 100)}%)\n")
    f.write(f"Mean area exceeding wind threshold of {threshold_2} m s⁻¹: {round(np.mean(mean_area_threshold_2))} km² of 1225 km² ({round(np.mean(mean_area_threshold_2) / 1225 * 100)}%)\n")
